pip installs

In [32]:
#pip install requests

imports

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.model_selection import train_test_split 
# from sklearn import datasets, tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, recall_score, precision_score
%matplotlib inline
import plotly.graph_objects as go
import plotly.io as pio  # me permite dibujar en un notebook
pio.renderers.default = "notebook"  # me permite dibujar en un notebook
import plotly.express as px
import plotly.figure_factory as ff
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import GridSearchCV
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools
import requests
import os
from collections import Counter

load/write labels and parameters

In [5]:

data_start_label='Parameter'
Parameter_data_rows = ['Unit','HighL','LowL','Tests#','Patterns']
HardBinDictName='HardBinName'
SoftBinDictName='SoftBinName'
header_rows_drop = 4
datatypes_per_unit = {
    'VOLTS':'float64',
    'Ohms':'float64',
    'V':'float64',
    'uSECONDS':'float64',
    'mAMPS':'float64',
    ' ':'float64',
    'code':'int64',
    'C':'float64',
    'LSB':'float64',
    'deg':'float64',
    'PF':'float64',
    'KHERTZ':'float64',
    'MHERTZ':'float64',
    'KkHz':'float64',
    '%':'float64',
    'uAMPS':'float64',
    'nnA':'float64',
    'nAMPS':'float64'}
testdatacols=[' SBIN','HBIN','DIE_X','DIE_Y','SITE','TIME','TOTAL_TESTS','LOT_ID','WAFER_ID']
confdict={'data_start_label':data_start_label,'Parameter_data_rows':Parameter_data_rows,'testdatacols':testdatacols,'HardBinDictName':HardBinDictName,'SoftBinDictName':SoftBinDictName,'datatypes_per_unit':datatypes_per_unit}



Load csv
generate df with data
generate HardBinDict
generate SoftBinDict
generate general_data

In [6]:
def load_data (_file_location, _confdict):
    _rawdf = pd.read_csv(_file_location)  # carga csv
    #future adding exceptions for cases like no string match as "Parameter" to handle and track code breakdown
    _data_start_col_value=(_rawdf.iloc[:,0]==_confdict.get('data_start_label')).idxmax()+1  # search for label in table data start
    _df = pd.read_csv(_file_location, header=_data_start_col_value, index_col=_confdict.get('data_start_label'))  # load table data from csv into df
    _df.drop(columns=_df.columns[-1], axis=1, inplace=True) #extra column without data

    #separate testdata and parameter data (might get input to make this optional)
    _dftestdata = _df[_confdict.get('testdatacols')]
    _df.drop(columns=_confdict.get('testdatacols'), inplace=True)
    _dfpardata=_df.loc[_confdict.get('Parameter_data_rows')]
    _df.drop(_confdict.get('Parameter_data_rows'), inplace=True)
    _df.reset_index(drop=True)
    #_dfpardata.loc['Unit'].replace(confdict.get('datatypes_per_unit'))

    # 
    _rawdf = _rawdf.head(_data_start_col_value-1)  # drop table data
    _rawdf = _rawdf.tail(-header_rows_drop)  # drop 4 rows from the header
    _HardBinDict = (_rawdf[_rawdf.iloc[:,0]==_confdict.get('HardBinDictName')].iloc[:,[1,2]])
    _HardBinDict = _HardBinDict.set_index(_HardBinDict.iloc[:, 0]).iloc[:,1]
    _HardBinDict = _HardBinDict.to_dict()
    _SoftBinDict = (_rawdf[_rawdf.iloc[:,0]==_confdict.get('SoftBinDictName')].iloc[:,[1,2]])
    _SoftBinDict = _SoftBinDict.set_index(_SoftBinDict.iloc[:, 0]).iloc[:,1]
    _SoftBinDict = _SoftBinDict.to_dict()
    _rawdf = _rawdf[(_rawdf.iloc[:,0]!=_confdict.get('HardBinDictName'))&(_rawdf.iloc[:,0]!=_confdict.get('SoftBinDictName'))]  # drop softbin and hardbin
    _rawdf.dropna(how='all')
    _general_data = _rawdf.iloc[:,[0,1]]  # global data extracted by position
    _general_data = _general_data.set_index(_general_data.iloc[:, 0]).iloc[:,1]
    _general_data = _general_data.to_dict()

    # input corrections
    _dfpardata['HSR_SMM ANGLE_D1']['Unit'] = 'deg'  # unit is "code" and should be "deg"
    _dfpardata['HSR_SMM ANGLE_D2']['Unit'] = 'deg'  # unit is "code" and should be "deg"
    _df = _df.replace(r'^\s*$', '99999999', regex=True)  # if a chip blows, no further testing is done and a space char is placed instead of a value, cannot use np.NaN for int dtypes

    _coldtypes=_dfpardata.loc['Unit'].replace(_confdict.get('datatypes_per_unit')).fillna('float64').to_dict()  # if no unit is stated defaults to float dtype as there are many non integers
    _df = _df.astype(_coldtypes)

    return (_df, _general_data,_dfpardata,_dftestdata, _HardBinDict, _SoftBinDict)

In [7]:
file_location1 = "C:/Users/ggrinberg/Downloads/QR6129X1.csv"
file_location2 = "C:/Users/ggrinberg/Downloads/QH6129X2.csv"
file_location3 = "C:/Users/ggrinberg/Downloads/QR6129X3.csv"
file_location4 = "C:/Users/ggrinberg/Downloads/QH6129X4.csv"
(df1, general_data1 ,dfpardata1, dftestdata1, HardBinDict, SoftBinDict) = load_data (file_location1, confdict)
(df2, general_data2 ,dfpardata2, dftestdata2, HardBinDict, SoftBinDict) = load_data (file_location2, confdict)
(df3, general_data3 ,dfpardata3, dftestdata3, HardBinDict, SoftBinDict) = load_data (file_location3, confdict)
(df4, general_data4 ,dfpardata4, dftestdata4, HardBinDict, SoftBinDict) = load_data (file_location4, confdict)

In [43]:
print(f"blank cells filled with{df3['SENT Trigger Threshold L Code 0_D1']['PID-120']}")  # check how an empty cell is filled
print(f"parameter data are not equal among different tests as shown here:{dfpardata1.equals(dfpardata2)}")  # values and size not equal
measures = Counter(dfpardata1.columns.to_list()+dfpardata2.columns.to_list()+dfpardata3.columns.to_list()+dfpardata4.columns.to_list())
result = [_key for _key, _value in measures.items () if _value != 4]
print(f"there are {len(result)} of {len(measures)} measurements which are not in the 4 datasets, most are temperature related, check with pato ")

blank cells filled with99999999.0
parameter data are not equal among different tests as shown here:False
there are 87 of 862 measurements which are not in the 4 datasets, most are temperature related, check with pato 


In [114]:
Lims1=dfpardata1.loc[['LowL','HighL']].astype('float')
faulted_by_param = (df1.gt(Lims1.loc['HighL'], axis=1) | df1.lt(Lims1.loc['LowL'], axis=1))
print('mediciones fuera de limites para df1')
print(faulted_by_param.sum()[faulted_by_param.any()].sort_values(ascending=False).head(10))  # secondary sensor has ill defined thresholds

mediciones fuera de limites para df1
Secondary Temp Sensor (25C)_D1    136
Secondary Temp Sensor (25C)_D2    136
Primary Temp Sensor (25C)_D2        6
HSR_SMM_D2                          5
HSR_SMM_D1                          4
dtype: int64


In [115]:
Lims2=dfpardata2.loc[['LowL','HighL']].astype('float')
faulted_by_param = (df2.gt(Lims2.loc['HighL'], axis=1) | df2.lt(Lims2.loc['LowL'], axis=1))
print('mediciones fuera de limites para df2')
print(faulted_by_param.sum()[faulted_by_param.any()].sort_values(ascending=False).head(10))  # secondary sensor has ill defined thresholds

mediciones fuera de limites para df2
IWRevZen_D2    69
dtype: int64


In [116]:
Lims3=dfpardata3.loc[['LowL','HighL']].astype('float')
faulted_by_param = (df3.gt(Lims3.loc['HighL'], axis=1) | df3.lt(Lims3.loc['LowL'], axis=1))
print('mediciones fuera de limites para df3')
print(faulted_by_param.sum()[faulted_by_param.any()].sort_values(ascending=False).head(10))  # secondary sensor has ill defined thresholds

mediciones fuera de limites para df3
Secondary Temp Sensor (25C)_D1    126
Secondary Temp Sensor (25C)_D2    126
HSR_SMM_D2                          5
HSR_SMM_D1                          4
Primary Temp Sensor (25C)_D1        2
Intra Channel HP3 Fine_D1.1         2
Intra Channel HP5 Fine_D1.1         2
Intra Channel HP4 Coarse_D2.1       2
Intra Channel HP4 Coarse_D1.1       2
Intra Channel HP4 Fine_D2.1         2
dtype: int64


In [117]:
Lims4=dfpardata4.loc[['LowL','HighL']].astype('float')
faulted_by_param = (df4.gt(Lims4.loc['HighL'], axis=1) | df4.lt(Lims4.loc['LowL'], axis=1))
print('mediciones fuera de limites para df4')
print(faulted_by_param.sum().sort_values(ascending=False).head(10)) 

mediciones fuera de limites para df4
IWRevZen_D2                      64
Intra Channel HP1 Coarse_D2.1     2
Intra Channel HP4 Fine_D2.1       2
Intra Channel HP4 Fine_D1.1       2
Intra Channel HP3 Coarse_D2.1     2
Intra Channel HP3 Coarse_D1.1     2
Intra Channel HP3 Fine_D2.1       2
Intra Channel HP3 Fine_D1.1       2
Intra Channel HP2 Coarse_D2.1     2
Intra Channel HP2 Coarse_D1.1     2
dtype: int64
